## R script for OMOP to load Care Site data

### Setup

In [1]:
library(DBI)

# make db connection
# for format of database file see db/env/db_conf.txt
db <- read.delim( '../../db/env/feta.txt', header=TRUE, sep=' ' )
con <- dbConnect(RPostgres::Postgres(),
                 dbname = toString(db$database[1]),
                 host = toString(db$host[1]),
                 port = 5432,
                 user = toString(db$user),
                 password = toString(db$pass))

# check the connection
dbListTables(con)

[1] "geography_columns"               "geometry_columns"               
 [3] "spatial_ref_sys"                 "acs_income_bmi"                 
 [5] "attr_epa_aqi_2019"               "attr_florida_acs"               
 [7] "data_source"                     "care_site"                      
 [9] "condition_occurrence"            "attr_dindex"                    
[11] "attr_florida_tri_2018"           "dindex_bmi"                     
[13] "geo_florida_tri_2018"            "geo_miamidade_census_tract_2018"
[15] "measurement"                     "md_hexbins_1km"                 
[17] "geo_us_counties"                 "locations"                      
[19] "utm_grid"                        "persons"                        
[21] "visit_occurrence"                "sites"                          
[23] "attr_index"                      "geo_index"                      
[25] "site_history"                    "location_history"               
[27] "geo_point_template"              "$1"

## Load Site data

In [3]:
sitedata <- read.csv('../../../../data/uhealth-locations_clean.csv',header=FALSE)

In [5]:
for (i in 1:nrow(sitedata)) {
    
    # create location with geometry
    sql_createsite <- 'INSERT INTO sites ("site_id","latitude","longitude","geom_wgs84") 
                       VALUES (DEFAULT,$1,$2,ST_SetSRID(ST_MakePoint($2,$1),4326))
                       RETURNING site_id'
    res <- dbSendQuery(con,sql_createsite)
    dbBind(res, list(sitedata[i,2],sitedata[i,3]))
    site_ID <- dbFetch(res)[1,1]
    dbClearResult(res)

    # create care site record
    sql_createcaresite <- 'insert into care_site ("care_site_id","care_site_name")
                       values(DEFAULT,$1)
                       returning care_site_id;'
    res <- dbSendQuery(con,sql_createcaresite)
    dbBind(res, list(sitedata[i,1]))
    care_site_ID <- dbFetch(res)[1,1]
    dbClearResult(res)
    
    # link care site and location
    sql_createsitehist = 'INSERT INTO site_history ("site_id","entity_id")
                          VALUES ($1,$2);'
    res <- dbSendQuery(con,sql_createsitehist)
    dbBind(res, list(site_ID,care_site_ID))
    dbClearResult(res)

    print (paste(sitedata[i,1],sitedata[i,2],sitedata[i,3]))

}

[1] "UMHC SCCC 25.788502 -80.214651"
[1] "UMHC 25.790406 -80.211993"
[1] "ABLEH 25.790772 -80.21039"
[1] "UMH 25.790203 -80.212097"
[1] "UMMG PAC 25.787552 -80.213964"
[1] "UMMG KENDALL 25.686414 -80.350651"
[1] "UMHC DEERFIELD 26.301083 -80.122183"
[1] "UMHC CLINICAL RESEARCH BUILDING 25.787001 -80.212848"
[1] "UMHC SYL KENDALL 25.686438 -80.350639"
[1] "UMMG BPEI PALM BEACH 26.838451 -80.13247"
[1] "UMHC SYL CROSSROADS 26.108332 -80.255741"
[1] "UMMG AT UMHC 25.788457 -80.216276"
[1] "UMMG MAILMAN CENTER 25.790203 -80.214418"
[1] "UMMG DIABETES RESEARCH INSTITUTE 25.788906 -80.21143"
[1] "UMMG BOCA RATON 26.38382 -80.097239"
[1] "UMHC LENNAR 25.713529 -80.280007"
[1] "UMMG PALM BEACH GARDEN 26.845715 -80.08947"
[1] "UMHC SCCC,UMHC SCCC 25.788502 -80.214651"
[1] "ABLEH NAPLES 26.193074 -81.799586"
[1] "UMMG DEERING MEDICAL PLAZA 25.629317 -80.345964"
[1] "UMHC SYL CORAL SPRINGS 26.256831 -80.235316"
[1] "UMHC,UMHC 25.790406 -80.211993"
[1] "UMMG LENNAR 25.713529 -80.280007"
[1] "UMHC 